# Task 1 Algorithmic Analysis K-Means Clustering with Real World Dataset

# Name: Vinayak Ramachandra Bhosale

# Question 1

In [1]:
import pandas as pd

data = pd.read_csv("data.csv",header=None).values
label = pd.read_csv("label.csv",header=None).values.reshape(-1)

In [2]:
data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [3]:
label

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [4]:
import numpy as np
import random

In [5]:
def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

In [6]:
def cosine_similarity(a, b):
    return 1 - np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [7]:
def jaccard_similarity(a, b):
    return 1 - np.sum(np.minimum(a, b)) / np.sum(np.maximum(a, b))

In [8]:
def k_means(data, k, similarity_func, max_iterations=500):
    centroids = random.sample(list(data), k)
    iterations = 0
    old_sse = 0
    while True:
        # Assign each data point to its closest centroid
        clusters = [[] for i in range(k)]
        for point in data:
            distances = [similarity_func(point, centroid) for centroid in centroids]
            cluster_index = np.argmin(distances)
            clusters[cluster_index].append(point)
        
        # Calculate the new centroid for each cluster
        new_centroids = []
        sse = 0
        for i in range(k):
            cluster = clusters[i]
            if len(cluster) == 0:
                new_centroids.append(centroids[i])
                continue
            centroid = np.mean(cluster, axis=0)
            new_centroids.append(centroid)
            sse += np.sum([similarity_func(point, centroid) for point in cluster])
        
        # Check for convergence
        if np.allclose(new_centroids, centroids) or sse >= old_sse or iterations >= max_iterations:
            break
        
        # Update the centroids and SSE
        centroids = new_centroids
        old_sse = sse
        iterations += 1
    
    return clusters, centroids, sse

In [9]:
k = len(np.unique(label))
k

10

In [10]:
clusters_euclidean, centroids_euclidean, sse_euclidean = k_means(data, k, euclidean_distance)

clusters_cosine, centroids_cosine, sse_cosine = k_means(data, k, cosine_similarity)

clusters_jaccard, centroids_jaccard, sse_jaccard = k_means(data, k, jaccard_similarity)

In [11]:
print("SSE with Euclidean distance: ", sse_euclidean)
print("SSE with Cosine similarity: ", sse_cosine)
print("SSE with Jaccard similarity: ", sse_jaccard)

SSE with Euclidean distance:  16571095.936437659
SSE with Cosine similarity:  2828.834382198499
SSE with Jaccard similarity:  6411.21656112987


# Question 2

In [12]:
import numpy as np

class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
    
    def fit(self, X):
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = [self.distance_metric(x, c) for c in self.centroids]
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            new_centroids = []
            for cluster in clusters:
                if len(cluster) == 0:
                    new_centroids.append(np.zeros(X.shape[1]))
                else:
                    new_centroids.append(np.mean(cluster, axis=0))
            if np.allclose(self.centroids, new_centroids):
                break
            self.centroids = new_centroids
    
    def predict(self, X):
        distances = np.array([np.array([self.distance_metric(x, c) for c in self.centroids]) for x in X])
        return np.argmin(distances, axis=1)


In [20]:
km_eu = KMeans(k=10, max_iter=100, distance_metric=euclidean_distance)
km_eu.fit(data)
eu_predictions = km_eu.predict(data)

In [14]:
km_cs = KMeans(k=10, max_iter=100, distance_metric=cosine_similarity)
km_cs.fit(data)
cs_predictions = km_cs.predict(data)

In [15]:
km_js = KMeans(k=10, max_iter=100, distance_metric=jaccard_similarity)
km_js.fit(data)
js_predictions = km_js.predict(data)

In [16]:
def Accuracy(predicted, actual):
    count = 0
    total = len(label)
    for i in range(total):
        if predicted[i] == actual[i]:
            count += 1
    return (count/total)*100

In [21]:
print("Accuracy of Euclidean Distance kmeans:",Accuracy(eu_predictions,label))
print("Accuracy of cosine similarity kmeans:",Accuracy(cs_predictions,label))
print("Accuracy of jaccard similarity kmeans:",Accuracy(js_predictions,label))

Accuracy of Euclidean Distance kmeans: 15.989999999999998
Accuracy of cosine similarity kmeans: 11.73
Accuracy of jaccard similarity kmeans: 7.55


# Question 3

In [49]:
import pandas as pd
import numpy as np
import random

data = pd.read_csv("data.csv",header=None).values
label = pd.read_csv("label.csv",header=None).values.reshape(-1)


def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

def cosine_similarity(a, b):
    return 1 - np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def jaccard_similarity(a, b):
    return 1 - np.sum(np.minimum(a, b)) / np.sum(np.maximum(a, b))

k = len(np.unique(label))

In [8]:
import numpy as np
import time

class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance, seed=42):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
        self.seed = seed
    
    def fit(self, X):
        np.random.seed(self.seed)
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_sse = np.inf
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = [self.distance_metric(x, c) for c in self.centroids]
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            new_centroids = []
            for cluster in clusters:
                if len(cluster) == 0:
                    new_centroids.append(np.zeros(X.shape[1]))
                else:
                    new_centroids.append(np.mean(cluster, axis=0))
            sse = np.sum([np.sum((cluster - centroid)**2) for cluster, centroid in zip(clusters, new_centroids)])
            if np.allclose(self.centroids, new_centroids) or sse > prev_sse:
                break
            self.centroids = new_centroids
            prev_sse = sse
    
    def predict(self, X):
        distances = np.array([np.array([self.distance_metric(x, c) for c in self.centroids]) for x in X])
        return np.argmin(distances, axis=1)


t1 = time.time()
kmeans_euclidean = KMeans(k=10, max_iter=500, distance_metric=euclidean_distance)
kmeans_euclidean.fit(data)
t2 = time.time()
print("Euclidean-K-means converged in {} iterations".format(len(kmeans_euclidean.centroids)))
print("Time taken by Euclidean-K-means to converge is {}".format(t2-t1))


Euclidean-K-means converged in 10 iterations
Time taken by Euclidean-K-means to converge is 36.97478365898132


In [9]:
t3 = time.time()
kmeans_cosine = KMeans(k=10, max_iter=500, distance_metric=cosine_similarity)
kmeans_cosine.fit(data)
t4 = time.time()
print("Cosine-K-means converged in {} iterations".format(len(kmeans_cosine.centroids)))
print("Time taken by Cosine-K-means to converge is {}".format(t4-t3))




Cosine-K-means converged in 10 iterations
Time taken by Cosine-K-means to converge is 63.50847101211548


In [10]:
t5 = time.time()
kmeans_jaccard = KMeans(k=10, max_iter=500, distance_metric=jaccard_similarity)
kmeans_jaccard.fit(data)
t6 = time.time()
print("Jaccard-K-means converged in {} iterations".format(len(kmeans_jaccard.centroids)))
print("Time taken by Jaccard-K-means to converge is {}".format(t6-t5))

Jaccard-K-means converged in 10 iterations
Time taken by Jaccard-K-means to converge is 88.04422998428345


# Question 4

when there is no change in centroid position

In [19]:
import numpy as np
from scipy.spatial.distance import cdist

# Define KMeans class
class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance, seed=42):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
        self.seed = seed

    def fit(self, X):
        np.random.seed(self.seed)
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_centroids = None
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = cdist(x.reshape(1, -1), self.centroids, metric=self.distance_metric)
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            if np.allclose(prev_centroids, self.centroids):
                break

    def predict(self, X):
        distances = cdist(X, self.centroids, metric=self.distance_metric)
        return np.argmin(distances, axis=1)

# Generate sample data
X, y = data,label

# Initialize KMeans models
euclidean_kmeans = KMeans(k=10, max_iter=100, distance_metric=euclidean_distance, seed=42)
cosine_kmeans = KMeans(k=10, max_iter=100, distance_metric=cosine_similarity, seed=42)
jaccard_kmeans = KMeans(k=10, max_iter=100, distance_metric=jaccard_similarity, seed=42)

In [20]:
euclidean_kmeans.fit(X)
euclidean_sse = np.sum(np.min(cdist(X, euclidean_kmeans.centroids, 'euclidean'), axis=1)**2)
print("Euclidean-K-means SSE:", euclidean_sse)

Euclidean-K-means SSE: 25414767689.961178


In [21]:
cosine_kmeans.fit(X)
cosine_sse = np.sum(np.min(cdist(X, cosine_kmeans.centroids, 'cosine'), axis=1)**2)
print("Cosine-K-means SSE:", cosine_sse)

Cosine-K-means SSE: 686.4355725684936


In [22]:
jaccard_kmeans.fit(X)
jaccard_sse = np.sum(np.min(cdist(X, jaccard_kmeans.centroids, 'jaccard'), axis=1)**2)
print("Jaccard-K-means SSE:", jaccard_sse)

Jaccard-K-means SSE: 9999.83773073805


when the SSE value increases in the next iteration

In [33]:
class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance, seed=42):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
        self.seed = seed

    def fit(self, X):
        np.random.seed(self.seed)
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_centroids = None
        prev_sse = float('inf')
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = cdist(x.reshape(1, -1), self.centroids, metric=self.distance_metric)
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            sse = np.sum([np.sum(cdist(cluster, centroid.reshape(1,-1), metric=self.distance_metric)**2) for cluster, centroid in zip(clusters, self.centroids)])
            if sse > prev_sse:
                break
            prev_sse = sse
            if np.allclose(prev_centroids, self.centroids):
                break

    def predict(self, X):
        distances = cdist(X, self.centroids, metric=self.distance_metric)
        return np.argmin(distances, axis=1)

X, y = data,label

# Initialize KMeans models
euclidean_kmeans = KMeans(k=10, max_iter=100, distance_metric=euclidean_distance, seed=42)
cosine_kmeans = KMeans(k=10, max_iter=100, distance_metric=cosine_similarity, seed=42)
jaccard_kmeans = KMeans(k=10, max_iter=100, distance_metric=jaccard_similarity, seed=42)

In [34]:
euclidean_kmeans.fit(X)
euclidean_sse = np.sum(np.min(cdist(X, euclidean_kmeans.centroids, 'euclidean'), axis=1)**2)
print("Euclidean-K-means SSE:", euclidean_sse)

Euclidean-K-means SSE: 25414767689.961178


In [35]:
cosine_kmeans.fit(X)
cosine_sse = np.sum(np.min(cdist(X, cosine_kmeans.centroids, 'cosine'), axis=1)**2)
print("Cosine-K-means SSE:", cosine_sse)

Cosine-K-means SSE: 686.229294287177


In [36]:
jaccard_kmeans.fit(X)
jaccard_sse = np.sum(np.min(cdist(X, jaccard_kmeans.centroids, 'jaccard'), axis=1)**2)
print("Jaccard-K-means SSE:", jaccard_sse)

Jaccard-K-means SSE: 9999.907741263536


In [51]:
import numpy as np
from scipy.spatial.distance import cdist



# Define KMeans class
class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance, seed=42):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
        self.seed = seed

    def fit(self, X):
        np.random.seed(self.seed)
        self.centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_centroids = None
        for i in range(self.max_iter):
            clusters = [[] for _ in range(self.k)]
            for x in X:
                distances = cdist(x.reshape(1, -1), self.centroids, metric=self.distance_metric)
                cluster = np.argmin(distances)
                clusters[cluster].append(x)
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            if np.allclose(prev_centroids, self.centroids):
                break
            elif i == self.max_iter-1:
                print(f"KMeans did not converge in {self.max_iter} iterations")
                break

    def predict(self, X):
        distances = cdist(X, self.centroids, metric=self.distance_metric)
        return np.argmin(distances, axis=1)

# Generate sample data
X, y = data,label

# Initialize KMeans models
euclidean_kmeans = KMeans(k=10, max_iter=100, distance_metric=euclidean_distance, seed=42)
cosine_kmeans = KMeans(k=10, max_iter=100, distance_metric=cosine_similarity, seed=42)
jaccard_kmeans = KMeans(k=10, max_iter=100, distance_metric=jaccard_similarity, seed=42)

# Fit the models on the data
euclidean_kmeans.fit(X)
cosine_kmeans.fit(X)
jaccard_kmeans.fit(X)

# Calculate the SSEs for each model
euclidean_sse = sum(np.min(cdist(X, euclidean_kmeans.centroids, euclidean_distance), axis=1)**2)
cosine_sse = sum(np.min(cdist(X, cosine_kmeans.centroids, cosine_similarity), axis=1)**2)
jaccard_sse = sum(np.min(cdist(X, jaccard_kmeans.centroids, jaccard_similarity), axis=1)**2)

# Print the SSEs for each model
print("Euclidean-K-means SSE:", euclidean_sse)
print("Cosine-K-means SSE:", cosine_sse)
print("Jaccard-K-means SSE:", jaccard_sse)

Euclidean-K-means SSE: 25414767689.9611
Cosine-K-means SSE: 686.435572568491
Jaccard-K-means SSE: 3660.389493716567
